In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
# Selenium 드라이버 실행
options = webdriver.ChromeOptions()
options.add_argument('headless')  # Chrome 창 숨기기 옵션
driver = webdriver.Chrome('/Users/parkjunhyeong/Downloads/chromedriver', options=options)

# 네이버 증권 뉴스 페이지로 이동
driver.get('https://finance.naver.com/news/mainnews.naver')

# 페이지 로딩을 위해 3초 대기
time.sleep(3)

# 페이지 소스코드 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 뉴스 제목 추출하기
src = soup.select('.articleSubject')
news_titles  = [news.text for news in src]
news_titles = [news.replace("\n", "") for news in news_titles]


#뉴스 링크 추출하기
news_links = []
base_url = 'https://finance.naver.com/'

for s in src:
    end = str(s.a).find(">")
    link = str(s.a)[9:end-1]
    link = base_url + link.replace("amp;","")
    news_links.append(link)

#뉴스 내용 추출하기
news_contents = []
for link in news_links:
    driver.get(link)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.select('.articleCont')[0].text.strip()
    news_contents.append(content)

# 드라이버 종료
driver.quit()

/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5593/1994811183.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/parkjunhyeong/Downloads/chromedriver', options=options)


In [3]:
df = pd.DataFrame(columns = ['제목', '링크', '본문'])
df['제목'] = news_titles
df['링크'] = news_links
df['본문'] = news_contents

df

,제목,링크,본문
0,'SG 사태' 잠잠해지지만...금융 위기 리스크 여전 [주간증시전망],https://finance.naver.com//news/news_read.nave...,"코스피지수가 전날보다 0.02% 내린 2,500.94로 집계된 지난 4일 오후 서울..."
1,[이번주 증시] 실적 시즌 본격화에 종목간 차별화 전망,https://finance.naver.com//news/news_read.nave...,코스피 예상 밴드 2420~2550이번주 증시는 상장사들의 양호한 1분기 실적에 힘...
2,[펀드와치]르세라핌·에스파 줄컴백…춤추는 K팝 ETF,https://finance.naver.com//news/news_read.nave...,주식형 펀드 국내 +1.02% 해외 +1.63%엔터사 담은 K팝·메타버스 ETF 수...
3,美금리인상 넘은 증시…실적발표 속 변동성장세[주간증시전망],https://finance.naver.com//news/news_read.nave...,"베이비스텝 밟은 FOMC, 시장은 ""금리인상 끝""美 CPI 발표·부채한도 협상에 주..."


In [4]:
df.to_excel('네이버 뉴스.xlsx')